## Model

In [37]:
import pandas as pd
import numpy as np
import time
from geopy.geocoders import Nominatim
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
import optuna
from scipy.optimize import minimize
import lightgbm as lgb
from catboost import CatBoostRegressor


In [38]:
# ============================================================================
# SECTION 1: CACHE ZONE COORDINATES
# ============================================================================
def cache_zone_coordinates(train_csv="train.csv", output_csv="zone_coordinates.csv"):
    df = pd.read_csv(train_csv)
    zones = df['zone'].dropna().unique()
    geolocator = Nominatim(user_agent="milan-rent-predictor")
    zone_coords = []

    manual_coords = {
    'bignami - ponale': (45.52474182295783, 9.210809331754353),
    'plebisciti - susa': (45.46981485298878, 9.222436710128576),
    'piave - tricolore': (45.467919604242574, 9.207210598762472),
    'cermenate - abbiategrasso': (45.437728368643256, 9.178810174203347),
    'monte rosa - lotto': (45.47660006699399, 9.147109899997002),
    'pezzotti - meda': (45.44162110101479, 9.181450432849504),
    'garibaldi - corso como': (45.482477141567884, 9.187283093211027),
    'martini - insubria': (45.46060849794086, 9.220424911760034),
    'navigli - darsena': (45.45233840157201, 9.177546326735703),
    'vercelli - wagner': (45.46676449725214, 9.157491657821524),
    'portello - parco vittoria': (45.48718781267403, 9.149662341091865),
    'amendola - buonarroti': (45.4722665381493, 9.152819453006444),
    'tre castelli - faenza': (45.43891003610351, 9.14256870703347),
    'ghisolfa - mac mahon': (45.49328695863271, 9.158754110405624),
    'cantalupa - san paolo': (45.42876257307274, 9.159224968271952),
    'borgogna - largo augusto': (45.46378926342732, 9.198137506546672)
}

    for zone in zones:
        if zone in manual_coords:
            lat, lon = manual_coords[zone]
            print(f"{zone} (manual): {lat}, {lon}")
        else:
            try:
                loc = geolocator.geocode(f"{zone}, Milan, Italy")
                lat, lon = (loc.latitude, loc.longitude) if loc else (None, None)
            except Exception:
                lat, lon = (None, None)
            print(f"{zone} (auto): {lat}, {lon}")
            time.sleep(1.1)

        zone_coords.append({'zone': zone, 'lat': lat, 'lon': lon})

    pd.DataFrame(zone_coords).to_csv(output_csv, index=False)
    print(f"Coordinates cached to {output_csv}")

# Uncomment to run once:
# cache_zone_coordinates()

In [39]:
# ============================================================================
# SECTION 2: HELPER FUNCTIONS
# ============================================================================
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in km
    φ1, φ2 = np.radians(lat1), np.radians(lat2)
    Δφ = φ2 - φ1
    Δλ = np.radians(lon2 - lon1)
    a = np.sin(Δφ/2)**2 + np.cos(φ1)*np.cos(φ2)*np.sin(Δλ/2)**2
    return 2 * R * np.arcsin(np.sqrt(a))

def add_avg_rent_per_room(df: pd.DataFrame, avg_rent_per_room: dict) -> pd.DataFrame:
    df['avg_rent_per_room'] = df['zone_raw'].map(avg_rent_per_room)
    return df

def preprocess(df: pd.DataFrame, zone_coords_csv="zone_coordinates.csv") -> pd.DataFrame:
    # 1. Contract Type
    df[['contract', 'term']] = df['contract_type'].str.split('|', n=1, expand=True)
    df['term'] = df['term'].str.strip().fillna('standard')
    df = pd.concat([df, pd.get_dummies(df['term'], prefix='term')], axis=1)
    df.drop(['contract_type','contract','term'], axis=1, inplace=True)

    # 2. Availability
    df['available'] = df['availability'].str.contains('available', case=False, na=False).astype(int)
    df.drop('availability', axis=1, inplace=True)

    # 3. Room Counts
    df['bedrooms'] = df['description'].str.extract(r'(\d+)\s*bedrooms?').astype(float)
    df['bathrooms'] = df['description'].str.extract(r'(\d+)\s*bathrooms?').astype(float)
    df.drop('description', axis=1, inplace=True)

    # 4. Other Features
    df['other_features_clean'] = df['other_features'].str.lower().fillna('')
    for feat in ['garden','terrace','balcony','furnished']:
        df[f'has_{feat}'] = df['other_features_clean'].str.contains(fr'(?:^|\|)\s*{feat}\s*(?:\||$)').astype(int)
    df['has_concierge'] = df['other_features_clean'].str.contains('concierge').astype(int)
    df.drop(['other_features_clean','other_features'], axis=1, inplace=True)

    # 5. Floor
    floor_map = {'Ground floor':0,'Semi-basement':-1,'Mezzanine':0.5}
    def parse_floor(x):
        if pd.isna(x): return np.nan
        if x in floor_map: return floor_map[x]
        try: return float(x)
        except: return np.nan
    df['floor_num'] = df['floor'].apply(parse_floor)
    df.drop('floor', axis=1, inplace=True)

    # 6. Elevator
    df['has_elevator'] = (df['elevator']=='yes').astype(int)
    df.drop('elevator', axis=1, inplace=True)

    # 7. Energy Class
    df['energy'] = df['energy_efficiency_class'].str.lower().str.replace(',','').fillna('unknown')
    order = {c:i for i,c in enumerate(['a','b','c','d','e','f','g','unknown'])}
    df['energy_code'] = df['energy'].map(order)
    df.drop(['energy_efficiency_class','energy'], axis=1, inplace=True)

    # 8. Condo Fees
    df['condominium_fees'] = df['condominium_fees'].fillna(df['condominium_fees'].median())

    # 9. Drop conditions
    df.drop('conditions', axis=1, inplace=True)

    # 10. Distance from Duomo
    duomo_lat, duomo_lon = 45.464323341569326, 9.191851418745173
    zone_df = pd.read_csv(zone_coords_csv).rename(columns={"zone_name": "zone"}).set_index('zone')
    df['zone_lat'] = df['zone'].map(zone_df['lat'])
    df['zone_lon'] = df['zone'].map(zone_df['lon'])
    df['dist_duomo_km'] = df.apply(
        lambda r: haversine(duomo_lat, duomo_lon, r.zone_lat, r.zone_lon)
        if pd.notna(r.zone_lat) else np.nan, axis=1
    )
    df['zone'] = df['dist_duomo_km']
    df.drop(['zone_lat','zone_lon','dist_duomo_km'], axis=1, inplace=True)

    # 11. Impute remaining numerics
    for c in ['bedrooms','bathrooms','floor_num','energy_code']:
        df[c] = df[c].fillna(df[c].median())

    return df

In [40]:
# ============================================================================
# SECTION 3: COMPUTE AVERAGE RENT PER ROOM
# ============================================================================
# Load raw train to compute averages
train_raw = pd.read_csv("train.csv")
train_raw["zone_raw"] = train_raw["zone"]
train_raw["bedrooms"] = train_raw["description"].str.extract(r'(\d+)\s*bedrooms?').astype(float)

# Drop rows with missing y, zone, bedrooms=0
valid = train_raw.dropna(subset=["y","zone","bedrooms"])
valid = valid[valid["bedrooms"]!=0]

# Compute per-zone avg rent per bedroom
avg_rent_per_room = (
    valid.assign(rp=valid["y"]/valid["bedrooms"])
         .groupby("zone")["rp"]
         .mean()
         .to_dict()
)


In [41]:
import pandas as pd
import numpy as np
import optuna
import xgboost as xgb

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import BayesianRidge

# ============================================================================
# SECTION 4: TRAIN/VALID SPLIT & OPTUNA TUNING + STACKING WITH BAYESIANRidge
# ============================================================================

# Prepare full train for modeling
df = pd.read_csv("train.csv")
df["zone_raw"] = df["zone"]
df = add_avg_rent_per_room(df, avg_rent_per_room)
df_processed = preprocess(df)

X = df_processed.drop(columns=["y", "zone_raw", "w"])
y = df["y"]

# Optuna objective for tuning base XGBRegressor
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 1.0),
        "random_state": 42,
        "n_jobs": -1,
        "verbosity": 0,
    }
    # Use train/valid split for Optuna eval
    X_train, X_valid, y_train, y_valid = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

# Run Optuna to find best hyperparameters
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

best_params = study.best_params
print("Best params found by Optuna:", best_params)

# --- Stacking with 5-fold CV ---

kf = KFold(n_splits=5, shuffle=True, random_state=42)
oof_preds = np.zeros(len(X))
test_preds = []  # for possible test set predictions stacking (optional)

# Initialize base model with best hyperparams from tuning
base_model_params = best_params.copy()
base_model_params.update({"random_state": 42, "n_jobs": -1, "verbosity": 0})

for fold, (train_idx, valid_idx) in enumerate(kf.split(X)):
    print(f"Training fold {fold+1}...")
    X_train_fold, X_valid_fold = X.iloc[train_idx], X.iloc[valid_idx]
    y_train_fold, y_valid_fold = y.iloc[train_idx], y.iloc[valid_idx]

    model = xgb.XGBRegressor(**base_model_params)
    model.fit(X_train_fold, y_train_fold)
    preds_valid = model.predict(X_valid_fold)
    oof_preds[valid_idx] = preds_valid

    fold_mae = mean_absolute_error(y_valid_fold, preds_valid)
    print(f"Fold {fold+1} MAE: {fold_mae:.4f}")

print(f"OOF CV MAE: {mean_absolute_error(y, oof_preds):.4f}")

# Fit Bayesian Ridge meta-model on out-of-fold predictions
meta_model = BayesianRidge()
meta_model.fit(oof_preds.reshape(-1, 1), y)

print("Stacking meta-model training complete.")
print("Stacking train MAE:", mean_absolute_error(y, meta_model.predict(oof_preds.reshape(-1, 1))))

# Train base model on full data for final stacking
final_base_model = xgb.XGBRegressor(**base_model_params)
final_base_model.fit(X, y)

# Save for next step: final_base_model and meta_model


[I 2025-05-21 17:13:42,563] A new study created in memory with name: no-name-81e664fb-c853-4455-a445-1de73338ec60
[I 2025-05-21 17:13:42,729] Trial 0 finished with value: 348.8016662597656 and parameters: {'n_estimators': 283, 'max_depth': 3, 'learning_rate': 0.13328527115550634, 'subsample': 0.9008835395151014, 'colsample_bytree': 0.8987660178632335, 'reg_alpha': 0.07473486637151838, 'reg_lambda': 0.1539378896490472}. Best is trial 0 with value: 348.8016662597656.
[I 2025-05-21 17:13:42,936] Trial 1 finished with value: 347.5910339355469 and parameters: {'n_estimators': 239, 'max_depth': 6, 'learning_rate': 0.03350610865298108, 'subsample': 0.7625009990378672, 'colsample_bytree': 0.6957382391210973, 'reg_alpha': 0.35933713669857914, 'reg_lambda': 0.5094744649703183}. Best is trial 1 with value: 347.5910339355469.
[I 2025-05-21 17:13:43,425] Trial 2 finished with value: 347.3284606933594 and parameters: {'n_estimators': 477, 'max_depth': 7, 'learning_rate': 0.011645860183605104, 'subsa

Best params found by Optuna: {'n_estimators': 996, 'max_depth': 9, 'learning_rate': 0.015570993688667937, 'subsample': 0.8824960837081263, 'colsample_bytree': 0.8273144911084909, 'reg_alpha': 0.7279894971557971, 'reg_lambda': 0.0023141769507849866}
Training fold 1...
Fold 1 MAE: 328.2542
Training fold 2...
Fold 2 MAE: 316.1053
Training fold 3...
Fold 3 MAE: 312.3821
Training fold 4...
Fold 4 MAE: 328.9317
Training fold 5...
Fold 5 MAE: 346.8823
OOF CV MAE: 326.5111
Stacking meta-model training complete.
Stacking train MAE: 326.35066129455856


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8273144911084909, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, feature_weights=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.015570993688667937,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=9, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=996, n_jobs=-1,
             num_parallel_tree=None, ...)

In [ ]:
# ============================================================================
# SECTION 5: PREDICT ON TEST SET & WRITE OUTPUT (STACKING)
# ============================================================================

# Load and preprocess test data
test = pd.read_csv("test.csv")
test["zone_raw"] = test["zone"]
test = add_avg_rent_per_room(test, avg_rent_per_room)
test_processed = preprocess(test)

# Drop 'zone_raw' to match training features
test_processed = test_processed.drop(columns=["zone_raw"])

# Predict with the base model first
base_preds = final_base_model.predict(test_processed)

# Use Bayesian Ridge meta-model to get final predictions
final_preds = meta_model.predict(base_preds.reshape(-1, 1))

# Save predictions to TXT file (single-column, no header)
pd.Series(final_preds).to_csv("predictions_final.txt", index=False, header=False)


✅ Predictions saved to predictions_final.txt
